In [1]:
import gron
import Oil
import Gas

In [2]:
#функция под конкретные исходные данные, для получения значений объем газа, плотность газа при p,T
def get_gas_parametrs (p,t, q_gas_std,relative_density_gas, y_a, y_c):
    """
    p - МПа,
    t - К,
    q_gas_std - суммарный расход газа в стандартных условиях выделившегося при p, t
    relative_density_gas - относительная плотность газа
    y_a, y_c - содержание азота, ув части в д.ед.
    
    volume_gas_p_t, density_gas_p_t - расход газа при [p,t], плотность газа при [p,t]
    """
    
    p=p*10**6
    relative_density_hydrocarbon = Gas._relative_density_hydrocarbon(relative_density_gas, y_a)
    p_priv = Gas._Lyapkov_pressure_priv(p, relative_density_hydrocarbon)
    t_priv = Gas._Lyapkov_temperature_priv(t, relative_density_hydrocarbon)
    z_c = Gas._z_y(p_priv,t_priv)
    z_a = Gas._z_a(p, t)
    z_supercompress = Gas._z_supercompress(z_c, y_c, z_a, y_a)
    density_gas_p_t = Gas._density_gas_p_T(p,t,z_supercompress,density_gas_std,T_0=294)
    volume_gas_p_t = Gas._Volume_gas_p_T(p,t,z_supercompress,q_gas_std,T_0=294)
    
    return {'q_gas_p_t':round(volume_gas_p_t,2),'density_gas_p_t':round(density_gas_p_t,3)}

In [3]:
def get_oil_parametrs(p, t, p_bbp, t_reservoir, gas_saturation, y_a, y_c1, density_oil_without_gas, relative_density_gas):
    
    #2
    pressure_bbp_T = Oil._pressure_bbp_T (p_bbp, t_reservoir, t, gas_saturation, y_a, y_c1)

    #3
    if p < pressure_bbp_T:
        R = Oil._R(p, pressure_bbp_T)
        m = Oil._m (t, density_oil_without_gas, relative_density_gas)
        D = Oil._D (t, density_oil_without_gas, relative_density_gas)
        volume_separate_gas = Oil._volume_separate_gas(gas_saturation, R,m,D)
    else:
        volume_separate_gas = 0


    #4
    if volume_separate_gas == 0:
        volume_dissolved_gas = gas_saturation
    else:
        volume_dissolved_gas = Oil._volume_dissolved_gas(gas_saturation, m, volume_separate_gas)
        if volume_dissolved_gas>gas_saturation:
            volume_dissolved_gas = gas_saturation

    #5
    a = Oil._a(t)
    u = Oil._u(density_oil_without_gas, gas_saturation)

    if volume_separate_gas == 0:
        relative_density_separate_gas = 0
    else: 
        relative_density_separate_gas = Oil._relative_density_separate_gas(relative_density_gas, a, u, R)

    #6
    m = Oil._m (t, density_oil_without_gas, relative_density_gas)
    relative_density_dissolved_gas = Oil._relative_density_dissolved_gas (gas_saturation, a, m, relative_density_gas, relative_density_separate_gas, volume_separate_gas, volume_dissolved_gas)

    #7
    alpha_n = Oil._alpha_n(density_oil_without_gas)
    lambda_T = Oil._lambda_T(density_oil_without_gas, relative_density_dissolved_gas, a, volume_dissolved_gas)
    b_oil = Oil._b_oil(p ,t ,density_oil_without_gas, volume_dissolved_gas, lambda_T, m, alpha_n)



    #8
    density_oil_with_gas = Oil._density_oil_with_gas(
        density_oil_without_gas,
        relative_density_dissolved_gas,
        volume_dissolved_gas,
        a,
        m,
        b_oil)

    #9 
    viscosity_without_gas, a_viscosity, b_viscosity = Oil._Dunyshkin_viscosity_without_gas (density_oil_without_gas),Oil._a_viscosity(t), Oil._b_viscosity (density_oil_without_gas, t)

    viscosity_without_gas_T = Oil._viscosity_without_gas_T(density_oil_without_gas, t, a_viscosity, b_viscosity, viscosity_without_gas)

    #10 
    volume_dissolved_prived = Oil._volume_dissolved_prived(volume_dissolved_gas, density_oil_without_gas, alpha_n)
    A_visc_dissolved = Oil._A_visc_dissolved(volume_dissolved_prived)
    B_visc_dissolved = Oil._B_visc_dissolved (volume_dissolved_prived)
    viscosity_oil_dissolved_gas = round(Oil._viscosity_dissolved_gas(
        A_visc_dissolved,
        B_visc_dissolved,
        viscosity_without_gas_T),4)

    #11
    sigma_oil_gas = Oil._sigma_oil_gas(p,t)
    return {'viscosity_oil_dissolved_gas':viscosity_oil_dissolved_gas, 
            'density_oil_with_gas':density_oil_with_gas, 
            'b_oil':b_oil, 
            'relative_density_separate_gas':relative_density_separate_gas, 
            'volume_separate_gas':volume_separate_gas}

_____________________________________
Исходные свойства <b>нефти, газа</b> информация по месторождению

In [4]:
#газосодрежание пластовой нефти, газовый фактор, м3/м3
gas_saturation = 56 
#давление насыщения, МПа
p_bbp = 9 
#плотность дегазированной нефти, кг/м3
density_oil_without_gas = 860
#плотность нефти в пластовых условиях, кг/м3
density_oil_reservoir = 800

#плотность воды, кг/м3
density_water = 1000

#плотность газа стандартные условия, кг/м3
density_gas_std = 1.45
#содержание в газе метана, %
y_c1 = 0.4
#содержание в газе азота, %
y_a = 0.08
#содержание в газе УВ части, %
y_c = 1-y_a-0.1
#пластовая температура, К
t_reservoir = 40+273

1. Исходные даные о тех.режиме скважины
2. Данные исследования КРД

Исследование - 5РМ
скважина 2Ф(D) 

In [5]:
#дебит по жидкости м3/сут
q_liquid = 40
#обводнённость, д.ед
B = 0
#диаметр экспл. колонны внутренний, м
D_casing_in = 0.133
#диаметр нкт колонны внутренний, м
D_tubing_in = 0.0503
#давление на устье в НКТ, МПа
p_head_tubing = 0.9
#Глубина скважины, м
h_well = 1605
#Глубина спуска труб, м
h_tubing = 1205


дополнительные параметры

In [6]:
#давление на забое , МПа
p_wf = 11.435
#пластовое давление, МПа
p_reservoir = round((h_well*9.81*1000/10**6 + 2),2)
#температура на устье, К
t_head_tubing = 294
# К продуктивности, м3/атм
PI = round(q_liquid/(p_reservoir - p_wf),3)
#молекулярная масса газа
mean_mol_mass = density_gas_std*24.05
#относительная плотность газа, кг/м3
relative_density_gas = Gas._relative_density(mean_mol_mass)

In [7]:
#данные исследований
h_test = [105, 305, 505, 705, 905, 1105, 1305, 1405, 1505, 1605]
p_test = [1.12, 1.83, 2.957, 4.355, 5.785, 7.3, 8.953, 9.863, 10.176, 11.435]

_____________________________________________________________
OIL p,T 

In [8]:
p, t = p_reservoir-10, t_reservoir-10
#определение диаметра подъёмника
D_tubing = D_tubing_in

In [9]:
def get_all_parametrs(p, t, p_bbp, t_reservoir, gas_saturation, y_a, y_c1, y_c, density_oil_without_gas, relative_density_gas, density_water, B, q_liquid, D_tubing):
    oil_parametrs = get_oil_parametrs(p, t, p_bbp, t_reservoir, gas_saturation, y_a, y_c1, density_oil_without_gas, relative_density_gas)

    viscosity_liquid_p_T = oil_parametrs['viscosity_oil_dissolved_gas']
    density_liquid_p_T = oil_parametrs['density_oil_with_gas']*(1-B) + density_water*B
    q_liquid_p_T = q_liquid*(1-B)*oil_parametrs['b_oil']+q_liquid*B
    #суммарный объём газа выделившегося в ст. условиях, м3
    q_gas_std = q_liquid*(1-B)*oil_parametrs['volume_separate_gas']
    #относительная плотность газа выделвшиегося при p,t
    relative_density_gas_p_t = oil_parametrs['relative_density_separate_gas']

    #расчёт параметров газа
    gas_parametrs = get_gas_parametrs (p,t, q_gas_std,relative_density_gas_p_t, y_a, y_c)
    q_gas_p_T, density_gas_p_T = gas_parametrs['q_gas_p_t'], gas_parametrs['density_gas_p_t']

    gron_parametrs = gron.Gron_parametrs(q_liquid_p_T, q_gas_p_T, density_gas_p_T, density_liquid_p_T, viscosity_liquid_p_T, D_tubing).return_dict()

    return oil_parametrs, gas_parametrs, gron_parametrs

In [12]:
all_parametrs = get_all_parametrs(1, t, p_bbp, t_reservoir, gas_saturation, y_a, y_c1, y_c, density_oil_without_gas, relative_density_gas, density_water, B, q_liquid, D_tubing)

In [16]:
1/all_parametrs[2]['gradient']

253.1797753376578